### Vanilla TensorFlow

In [ ]:
def shuffle_batch(X, y, batch_size): # not really needed for XOR
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [ ]:
import numpy as np
import tensorflow as tf

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()

n_inputs = 2
n_hidden = 3
n_outputs = 1

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.float32, shape=(None), name='y')

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        #stddev = 2 / np.sqrt(n_inputs)
        #init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        #W = tf.Variable(init, name="weights")
        #b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        W = tf.Variable(tf.random_uniform([n_inputs, n_neurons], -1, 1), name = "weights")
        b = tf.Variable(tf.zeros([n_neurons]), name = "biases")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else: return Z
        
        
Theta1 = tf.Variable(tf.random_uniform([2,3], -1, 1), name = "Theta1")
Theta2 = tf.Variable(tf.random_uniform([3,1], -1, 1), name = "Theta2")

Bias1 = tf.Variable(tf.zeros([3]), name = "Bias1")
Bias2 = tf.Variable(tf.zeros([1]), name = "Bias2")
        

with tf.name_scope('nn'):
    hidden = neuron_layer(X, n_hidden, name='hidden', activation=tf.nn.sigmoid)
    probs = neuron_layer(hidden, n_outputs, name='outputs', activation=tf.nn.sigmoid)

with tf.name_scope('loss'):
    mse_loss = tf.reduce_mean(tf.squared_difference(y, probs), name='loss')
    #bin_xentropy_loss = tf.reduce_mean(( (y * tf.log(probs)) + ((1 - y) * tf.log(1.0 - probs)) ) * -1)

learning_rate = 1

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mse_loss)

with tf.name_scope('eval'):
    correct = tf.equal(tf.greater_equal(probs,0.5), tf.cast(y,tf.bool))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
#saver = tf.train.Saver()

#n_epochs = 500
#batch_size = 4

X_train = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
]
y_train = [[0],[1],[1],[0]]

with tf.Session() as sess:
    init.run()
    for epoch in range(500):
        _, mse, acc = sess.run([training_op, mse_loss, accuracy], 
                               feed_dict={X: np.array(X_train), y: np.array(y_train)})
        print("mse: %.4f, accuracy: %.2f" % (mse, acc))
        

In [ ]:
import tensorflow as tf
import time

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()

x_ = tf.placeholder(tf.float32, shape=[4,2], name = 'x-input')
y_ = tf.placeholder(tf.float32, shape=[4,1], name = 'y-input')

Theta1 = tf.Variable(tf.random_uniform([2,3], -1, 1), name = "Theta1")
Theta2 = tf.Variable(tf.random_uniform([3,1], -1, 1), name = "Theta2")

Bias1 = tf.Variable(tf.zeros([3]), name = "Bias1")
Bias2 = tf.Variable(tf.zeros([1]), name = "Bias2")

with tf.name_scope("layer2") as scope:
	A2 = tf.tanh(tf.matmul(x_, Theta1) + Bias1)

with tf.name_scope("layer3") as scope:
	Hypothesis = tf.sigmoid(tf.matmul(A2, Theta2) + Bias2)

with tf.name_scope("cost") as scope:
	cost = tf.reduce_mean(( (y_ * tf.log(Hypothesis)) + 
		((1 - y_) * tf.log(1.0 - Hypothesis)) ) * -1)

with tf.name_scope("train") as scope:
	train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cost)

XOR_X = [[0,0],[0,1],[1,0],[1,1]]
XOR_Y = [[0],[1],[1],[0]]

init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)

for i in range(5000):
    sess.run(train_step, feed_dict={x_: XOR_X, y_: XOR_Y})
    if i % 100 == 0:
        print('Epoch %5d' % i, end='  ')
        #print('Hypothesis ', sess.run(Hypothesis, feed_dict={x_: XOR_X, y_: XOR_Y}))
        print('cost ', sess.run(cost, feed_dict={x_: XOR_X, y_: XOR_Y}))

In [1]:
import numpy as np
import tensorflow as tf

n_inputs = 2
n_hidden = 3
n_outputs = 1

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.float32, shape=(None), name='y')

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else: return Z

with tf.name_scope('nn'):
    hidden = neuron_layer(X, n_hidden, name='hidden', activation=tf.nn.sigmoid)
    prediction_probabilities = neuron_layer(hidden, n_outputs, name='outputs', activation=tf.nn.sigmoid)

with tf.name_scope('loss'):
    mse_loss = tf.reduce_mean(tf.squared_difference(y, prediction_probabilities), name='loss')    
    loss_summary = tf.summary.scalar('mse_loss', mse_loss)
    
learning_rate = 0.1

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mse_loss)

with tf.name_scope('eval'):
    correct = tf.equal(tf.greater_equal(prediction_probabilities,0.5), tf.cast(y,tf.bool))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

init = tf.global_variables_initializer()

X_train = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]
y_train = [[0],[1],[1],[0]]

with tf.Session() as sess:
    init.run()
    for epoch in range(500):
        _, mse, acc = sess.run([training_op, mse_loss, accuracy], 
                               feed_dict={X: np.array(X_train), y: np.array(y_train)})
        print("mse: %.4f, accuracy: %.2f" % (mse, acc))

mse: 0.2956, accuracy: 0.50
mse: 0.2694, accuracy: 0.50
mse: 0.2553, accuracy: 0.50
mse: 0.2526, accuracy: 0.75
mse: 0.2552, accuracy: 0.50
mse: 0.2573, accuracy: 0.50
mse: 0.2570, accuracy: 0.50
mse: 0.2545, accuracy: 0.50
mse: 0.2510, accuracy: 0.50
mse: 0.2474, accuracy: 0.50
mse: 0.2446, accuracy: 0.50
mse: 0.2427, accuracy: 0.75
mse: 0.2417, accuracy: 0.50
mse: 0.2411, accuracy: 0.50
mse: 0.2407, accuracy: 0.50
mse: 0.2399, accuracy: 0.50
mse: 0.2388, accuracy: 0.50
mse: 0.2371, accuracy: 0.75
mse: 0.2350, accuracy: 0.50
mse: 0.2325, accuracy: 0.50
mse: 0.2300, accuracy: 0.50
mse: 0.2273, accuracy: 0.50
mse: 0.2248, accuracy: 0.50
mse: 0.2222, accuracy: 0.50
mse: 0.2195, accuracy: 0.75
mse: 0.2167, accuracy: 0.75
mse: 0.2137, accuracy: 0.75
mse: 0.2103, accuracy: 0.75
mse: 0.2067, accuracy: 0.75
mse: 0.2028, accuracy: 0.75
mse: 0.1987, accuracy: 0.75
mse: 0.1945, accuracy: 0.50
mse: 0.1903, accuracy: 0.50
mse: 0.1862, accuracy: 0.50
mse: 0.1822, accuracy: 0.50
mse: 0.1783, accurac

# LIST OF TF TO DOS:
- minimal example (no classes, saves, etc)
- classes, batches w/ variable size
- saves, tb, summaries
- eager